In [1]:
from acfunctions import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.stats as astat
from solartwins import *
import pickle
from IPython.display import Image, display, HTML
import starspot as ss
import starspot.rotation_tools as rt
from starspot import sigma_clipping
import lightkurve as lk
from lightkurve import search_targetpixelfile, search_lightcurvefile
import math
import exoplanet
import eleanor
from astropy.coordinates import SkyCoord
from tess_kep_funct import *
from tqdm import tqdm
from scipy import misc
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
import warnings
warnings.filterwarnings("ignore")

/Users/ilanadoran/opt/anaconda3/lib/python3.7/site-packages/theano/configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=False)

# key 

short_tess_period = periods calculated using starspot's pdm period function

short_periods = calculated mcquillan periods under 13 days

tics_short_periods = tic ids of the mcquillan-determined short periods

ls_periods = calculated lombs-scargle tess periods from starspot

# importing data from TESS_Kep

In [3]:
#dataset with tess vs kepler IDs
tess_kep = pd.read_csv('data_summary.csv')
tess_short = tess_kep[tess_kep.tess_ffi == True]
cat = pd.read_csv('tics_kics_mcq_rot.txt', names=['tic_id', 'kic_id', 'prot'], delim_whitespace=True)

In [4]:
#importing McQ periods
headers = 'TIC ID', 'KIC ID', 'McQ Periods';  

with open("tics_kics_mcq_rot.txt") as file:
    rot_table = [line.split() for line in file.read().splitlines()]

widths = [max(len(value) for value in col)
for col in zip(*(rot_table + [headers]))]

formatting = '{:{widths[0]}}  {:{widths[1]}}  {:{widths[2]}}'
formatting.format(*headers, widths=widths)
for tics_kics_mcq in rot_table:
    formatting.format(*tics_kics_mcq, widths=widths)

In [5]:
# converting txt file data to arrays
tic_ids = []
mcq_periods = []

for index, row in enumerate(rot_table):
    tic_ids.append(int(rot_table[index][0]))
    mcq_periods.append(float(rot_table[index][2]))

In [6]:
# selecting shortest periods only (<13 days, half a tess cycle)
short_periods = []
tics_short_periods = []

for index, period in enumerate(mcq_periods):
    if period < 13: 
        short_periods.append(period)
        tics_short_periods.append(tic_ids[index])
        
#limiting to the first 60
short_periods = short_periods[0:61]
tics_short_periods = tics_short_periods[0:61]

In [7]:
with open('tess_period.data', 'rb') as filehandle:
    short_tess_period = pickle.load(filehandle) 

In [8]:
short_tess_period[0]

7.730630630630631

In [9]:
output_notebook()

Loading BokehJS ...

In [10]:
#calculating lombs-scargle tess periods from starspot
ls_period_no_interp = []

for each, star in enumerate(tics_short_periods):
    star_el = eleanor.Source(tic=star)
    data = eleanor.TargetData(star_el, do_psf=True, do_pca=True)
    q = data.quality == 0
    
    time = data.time[q]
    norm_flux = data.pca_flux[q]/np.nanmedian(data.pca_flux[q])
    flux_err = np.sqrt(norm_flux)

    #exo = exoplanet.lomb_scargle_estimator(time, norm_flux, yerr=flux_err)#, max_period= 15, max_peaks=2)
    #print(exo['peaks'][1]['period'])
    
    rotate = ss.RotationModel(time, norm_flux, flux_err)
    ls_period = rotate.ls_rotation()
    ls_period_no_interp.append(ls_period)
    
    #exo_ls.append(exo['peaks'][1]['period'])

No eleanor postcard has been made for your target (yet). Using TessCut instead.




100%|██████████| 1197/1197 [00:17<00:00, 68.64it/s]


No eleanor postcard has been made for your target (yet). Using TessCut instead.


100%|██████████| 1241/1241 [00:18<00:00, 68.39it/s]


No eleanor postcard has been made for your target (yet). Using TessCut instead.


...

100%|██████████| 1197/1197 [00:21<00:00, 54.49it/s]


No eleanor postcard has been made for your target (yet). Using TessCut instead.


100%|██████████| 1197/1197 [00:20<00:00, 57.12it/s]


No eleanor postcard has been made for your target (yet). Using TessCut instead.


100%|██████████| 1197/1197 [00:22<00:00, 54.25it/s]


In [29]:
with open('ls_period_no_interp.data', 'wb') as filehandle:
    pickle.dump(ls_period_no_interp, filehandle)

In [30]:
with open('ls_period_no_interp.data', 'rb') as filehandle:
    ls_period_no_interp = pickle.load(filehandle) 

with open('ls_periods', 'wb') as filehandle:
    pickle.dump(ls_periods, filehandle)

with open('exo_ls.data', 'wb') as filehandle:
    pickle.dump(exo_ls, filehandle)

with open('ls_periods.data', 'rb') as filehandle:
    ls_periods = pickle.load(filehandle) 

with open('exo_ls.data', 'rb') as filehandle:
    exo_ls = pickle.load(filehandle) 

exo_0 = exoplanet.lomb_scargle_estimator(time, norm_flux, yerr=flux_err)
plt.plot(1/(exo_0['periodogram'][0]), exo_0['periodogram'][1])
plt.title('Periodigram')

exo_0

exo_acf = [] 

for each, star in enumerate(tics_short_periods):
    star_el = eleanor.Source(tic=star)
    data = eleanor.TargetData(star_el, do_psf=True, do_pca=True)
    q = data.quality == 0
    
    time = data.time[q]
    norm_flux = data.pca_flux[q]/np.nanmedian(data.pca_flux[q])
    flux_err = norm_flux * 0.01
    
    ex_1 = exoplanet.autocorr_estimator(time, norm_flux, yerr=flux_err, oversample=2.0, smooth=2.0)
    
    if len(ex_1['peaks']) > 0: 
        exo_acf.append(ex_1['peaks'][0]['period'])
    else:
        exo_acf.append(0)  

with open('exo_acf.data', 'wb') as filehandle:
    pickle.dump(exo_acf, filehandle)

with open('exo_acf.data', 'rb') as filehandle:
    exo_acf = pickle.load(filehandle) 

In [14]:
x =  short_periods
y = short_tess_period

source = ColumnDataSource(data=dict(
    x =  short_periods,
    y = short_tess_period, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

pdm_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title="PDM Period vs McQuillan Periods",
   x_axis_label='McQuillan Period', y_axis_label='TESS PDM Period', plot_width=900)

pdm_plot.line(x, x, name = "y=x", color = 'lightsteelblue', line_dash="4 4", legend = "y=x", line_width = 2)
pdm_plot.circle('x', 'y', source = source, color="mediumvioletred", line_color=None)
pdm_plot.line(x, x2, legend="y=2x", line_width=1, color = 'khaki', line_dash="dotted")
pdm_plot.line(x, x_half, legend="y=0.5*x", line_width=1, color = 'rosybrown', line_dash="dotted")

pdm_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(pdm_plot)

In [15]:
#finding differences between PDM and McQ tess periods
differences_mcq_pdm = []

for indx, period in enumerate(short_tess_period):
    diff = np.abs(period - short_periods[indx])
    differences_mcq_pdm.append(diff)

In [16]:
x =  short_periods
y = differences_mcq_pdm

source = ColumnDataSource(data=dict(
    x = short_periods,
    y = differences_mcq_pdm, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]
x_0 = [i*0 for i in x]

pdm_diff = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 12], title="Differences Between McQ and PDM TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Difference Between Period Calculations', plot_width=900)

pdm_diff.line(x, x_0, name = "y=0", color = 'bisque', line_dash="4 4", legend = "y=0", line_width = 2)
pdm_diff.circle('x', 'y', source = source, color="mediumseagreen", line_color=None)


pdm_diff.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(pdm_diff)

In [17]:
x =  short_periods
y = exo_acf

source = ColumnDataSource(data=dict(
    x =  short_periods,
    y = exo_acf, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

exo_acf_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title="Exoplanet ACF vs McQuillan Periods",
   x_axis_label='McQuillan Period', y_axis_label='Exoplanet ACF Period', plot_width=900)

exo_acf_plot.line(x, x, name = "y=x", color = 'lightseagreen', line_dash="4 4", legend = "y=x", line_width = 2)
exo_acf_plot.circle('x', 'y', source = source, color="darkgreen", line_color=None)
exo_acf_plot.line(x, x2, legend="y=2x", line_width=1, color = 'mediumpurple', line_dash="dotted")
exo_acf_plot.line(x, x_half, legend="y=0.5*x", line_width=1, color = 'skyblue', line_dash="dotted")

exo_acf_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(exo_acf_plot)

In [18]:
#finding differences between Exoplanet ACF and McQ tess periods
differences_exo_acf = []

for indx, period in enumerate(exo_acf):
    diff = np.abs(period - short_periods[indx])
    differences_exo_acf.append(diff)

In [19]:
x =  short_periods
y = exo_acf

source = ColumnDataSource(data=dict(
    x = short_periods,
    y = exo_acf, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]
x_0 = [i*0 for i in x]

exo_ac_diff = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 9], title="Differences Between McQ and Exoplanet ACF TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Difference Between Period Calculations', plot_width=900)

exo_ac_diff.line(x, x_0, name = "y=0", color = 'lightgreen', line_dash="4 4", legend = "y=0", line_width = 2)
exo_ac_diff.circle('x', 'y', source = source, color="indigo", line_color=None)


exo_ac_diff.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(exo_ac_diff)

In [20]:
x =  short_periods
y = ls_periods

source = ColumnDataSource(data=dict(
    x =  short_periods,
    y = ls_periods, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

ls_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title="Lomb-Scargle vs McQuillan TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Lomb-Scargle Period', plot_width=900)

ls_plot.line(x, x, name = "y=x", color = 'palegreen', line_dash="4 4", legend = "y=x", line_width = 2)
ls_plot.circle('x', 'y', source = source, color="darkcyan", line_color=None)
ls_plot.line(x, x2, legend="y=2x", line_width=1, color = 'dimgray', line_dash="dotted")
ls_plot.line(x, x_half, legend="y=0.5*x", line_width=1, color = 'sienna', line_dash="dotted")

ls_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(ls_plot)

In [21]:
#finding differences between L-S and McQ tess periods
differences_mcq_ls = []

for indx, period in enumerate(ls_periods):
    diff = np.abs(period - short_periods[indx])
    differences_mcq_ls.append(diff)

In [22]:
x =  short_periods
y = differences_mcq_ls

source = ColumnDataSource(data=dict(
    x = short_periods,
    y = differences_mcq_ls, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]
x_0 = [i*0 for i in x]

ls_diff = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 12], title="Differences Between McQ and L-S TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Difference Between Period Calculations', plot_width=900)

ls_diff.line(x, x_0, name = "y=0", color = 'tomato', line_dash="4 4", legend = "y=0", line_width = 2)
ls_diff.circle('x', 'y', source = source, color="black", line_color=None)


ls_diff.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(ls_diff)

In [23]:
x =  short_periods
y = exo_ls

source = ColumnDataSource(data=dict(
    x =  short_periods,
    y = exo_ls, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]

exo_ls_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[0, 15], title="Exoplant Lomb-Scargle vs McQuillan TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Lomb-Scargle Period', plot_width=900)

exo_ls_plot.line(x, x, name = "y=x", color = 'plum', line_dash="4 4", legend = "y=x", line_width = 2)
exo_ls_plot.circle('x', 'y', source = source, color="purple", line_color=None)
exo_ls_plot.line(x, x2, legend="y=2x", line_width=1, color = 'orange', line_dash="dotted")
exo_ls_plot.line(x, x_half, legend="y=0.5*x", line_width=1, color = 'powderblue', line_dash="dotted")

exo_ls_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(exo_ls_plot)

In [24]:
#finding differences between Exo Lomb-Scargle and McQ tess period
differences_exo_ls_mcq = []

for indx, period in enumerate(short_periods):
    diff = np.abs(period - exo_ls[indx])
    differences_exo_ls_mcq.append(diff)

In [25]:
x =  short_periods
y = differences_exo_ls_mcq

source = ColumnDataSource(data=dict(
    x = short_periods,
    y = differences_exo_ls_mcq, 
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]
x_0 = [i*0 for i in x]

exo_diff_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 15], title="Differences Between Exo L-S and McQ TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Difference Between Period Calculations', plot_width=900)

exo_diff_plot.line(x, x_0, name = "y=0", color = 'slategrey', line_dash="4 4", legend = "y=0", line_width = 2)
exo_diff_plot.circle('x', 'y', source = source, color="mediumblue", line_color=None)


exo_diff_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess")] 

show(exo_diff_plot)

In [26]:
#finding differences between Lomb-Scargle and PDM calculated tess period
differences_ls_pdm = []

for indx, period in enumerate(short_tess_period):
    tess_diff = np.abs(period - ls_periods[indx])
    differences_ls_pdm.append(tess_diff)

In [27]:
x =  short_periods
y = differences_ls_pdm

source = ColumnDataSource(data=dict(
    x = short_periods,
    y = differences_ls_pdm, 
    kep = (cat['kic_id'])[0:61],
    tess = (cat['tic_id'])[0:61]))

x2 = [i*2 for i in x]
x_half = [float(i)*0.5 for i in x]
x_0 = [i*0 for i in x]

diff_plot = figure(
   tools="pan,box_zoom,reset,save,hover",
   y_range=[-1, 15], title="Differences Between L-S and PDM TESS Periods",
   x_axis_label='McQuillan Period', y_axis_label='Difference Between Period Calculations', plot_width=900)

diff_plot.line(x, x_0, name = "y=0", color = 'lightpink', line_dash="4 4", legend = "y=0", line_width = 2)
diff_plot.circle('x', 'y', source = source, color="darkred", line_color=None)


diff_plot.hover.tooltips = [
    ("index", "$index"), 
    ("(x,y)", "($x, $y)"), 
    ("TIC ID", "@tess"), 
    ("KIC ID", "@kep")] 

show(diff_plot)